# Car Make and Model Recognizer


## Importing Libraries

In [1]:
from keras.layers import Conv2D, MaxPooling2D, GlobalAveragePooling2D, Dropout, Flatten, Dense, BatchNormalization
from keras.models import Sequential
from keras.preprocessing.image import ImageDataGenerator
from sklearn.datasets import load_files       
from keras.utils import np_utils
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd 
from glob import glob

Using TensorFlow backend.


## Pre-Processing

### Data Augmentation
First ImageDataGenerator is used to set the parameters of Augmenting and specifying the validation set
flow_from_directory creates image generators with target sizes and batches sizes specified, categorical class mode means we are dealing with a classifying problem 

In [2]:
train_datagen = ImageDataGenerator(rescale=1./255,
                                   zoom_range=0.25,
                                   rotation_range = 90,
                                   horizontal_flip=True)
test_datagen = ImageDataGenerator(rescale=1./255,
                                 horizontal_flip = True)

train_data = train_datagen.flow_from_directory('My_Cars/train',
                                              target_size=(128,128),
                                              batch_size=32,
                                              class_mode='categorical')
test_data = test_datagen.flow_from_directory('My_Cars/test',
                                              target_size=(128,128),
                                              batch_size=32,
                                              class_mode='categorical')

Found 8144 images belonging to 196 classes.
Found 8041 images belonging to 196 classes.


## Model
3 layers were used in this model

In [3]:
model = Sequential()
#Building the model
#First Layer
model.add(Conv2D(filters=32, kernel_size=6,padding='same', input_shape=(128, 128, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=2,strides = 4))
model.add(BatchNormalization())
#model.add(Dropout(0.25))
#Second Layer
model.add(Conv2D(filters=64, kernel_size=6, activation='relu'))
model.add(MaxPooling2D(pool_size=2,strides = 4))
model.add(BatchNormalization())
#model.add(Dropout(0.25))
#Third Layer
model.add(Conv2D(filters=128, kernel_size=6, activation='relu'))
model.add(MaxPooling2D(pool_size=2,strides = 4))
model.add(Dropout(0.25))
model.add(BatchNormalization())
#Fourth Layer
# model.add(Conv2D(filters=256, kernel_size=2,padding='same', activation='relu'))
# model.add(MaxPooling2D(pool_size=2,strides = 4))
#model.add(Dropout(0.25))

#model.add(GlobalAveragePooling2D())
#Hidden Layer
model.add(Dense(512, activation = 'relu'))
model.add(BatchNormalization())
model.add(Dense(512, activation = 'relu'))
model.add(BatchNormalization())
#output layer
model.add(Flatten())
model.add(Dense(196, activation='softmax'))
#compile models
model.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['accuracy'])


Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


In [ ]:
#Show Architecture
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 128, 128, 32)      3488      
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 32, 32, 32)        0         
_________________________________________________________________
batch_normalization_1 (Batch (None, 32, 32, 32)        128       
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 27, 27, 64)        73792     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 7, 7, 64)          0         
_________________________________________________________________
batch_normalization_2 (Batch (None, 7, 7, 64)          256       
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 2, 2, 128)         295040    
__________

### Train

In [ ]:
from keras.callbacks import ModelCheckpoint  

### Number of epochs to train.

epochs = 30

### Checkpointer to save best the best weights.

checkpointer = ModelCheckpoint(filepath='Saved_Models/weights.best.from_scratch15.hdf5', 
                               verbose=1, save_best_only=True)
## Training the Model
## pass training samples generator
## steps per epochs = number of training samples / batch size
## number of epochs
## pass validation samples generator
## validation steps = number of validation samples / batch size
## pass checkpointer
Saving = model.fit_generator(train_data,
          steps_per_epoch=256,  
          epochs=epochs,
          verbose=1, 
          validation_data=test_data,
          validation_steps=251,
          callbacks=[checkpointer])
# model.fit(train_data, test_data,
#                   callbacks = [checkpointer],
#                   validation_split = 0.33,
#                   epochs=epochs,
#           batch_size = 20,
#           verbose=1)

Instructions for updating:
Use tf.cast instead.
Epoch 1/30
256/256 [==============================] - 164s 640ms/step - loss: 5.6335 - acc: 0.0112 - val_loss: 5.8675 - val_acc: 0.0139

Epoch 00001: val_loss improved from inf to 5.86751, saving model to Saved_Models/weights.best.from_scratch15.hdf5
Epoch 2/30
256/256 [==============================] - 158s 617ms/step - loss: 5.3233 - acc: 0.0153 - val_loss: 5.5289 - val_acc: 0.0131

Epoch 00002: val_loss improved from 5.86751 to 5.52890, saving model to Saved_Models/weights.best.from_scratch15.hdf5
Epoch 3/30
256/256 [==============================] - 156s 610ms/step - loss: 5.2065 - acc: 0.0172 - val_loss: 5.4188 - val_acc: 0.0160

Epoch 00003: val_loss improved from 5.52890 to 5.41883, saving model to Saved_Models/weights.best.from_scratch15.hdf5
Epoch 4/30
256/256 [==============================] - 159s 621ms/step - loss: 5.1162 - acc: 0.0260 - val_loss: 6.3457 - val_acc: 0.0167

Epoch 00004: val_loss did not improve from 5.41883
Epo

## Showing Results
Show top 31 entries

In [ ]:

vals = pd.DataFrame.from_dict(Saving.history)
vals = pd.concat([pd.Series(range(0,100),name='epochs'),vals],axis=1)
vals.head(n=31)

Showing graphs of the results

In [ ]:
import seaborn as sns
sns.set_style({'xtick.bottom':False,
               'ytick.left':False,
               'axes.spines.bottom': False,
               'axes.spines.left': False,
               'axes.spines.right': False,
               'axes.spines.top': False})

ig,(ax,ax1) = plt.subplots(nrows=2,ncols=1,figsize=(20,20))
sns.scatterplot(x='epochs',y='acc',data=vals,ax=ax,color='r')
sns.lineplot(x='epochs',y='val_acc',data=vals,ax=ax,color='g')
sns.scatterplot(x='epochs',y='loss',data=vals,ax=ax1,color='r')
sns.lineplot(x='epochs',y='val_loss',data=vals,ax=ax1,color='g')
ax.legend(labels=['Test Accuracy','Training Accuracy'])
ax1.legend(labels=['Test Loss','Training Loss'])

In [ ]:
# for i in range(n_iterations):
#         print('\r{}/{}'. format(i, n_iterations))
#         #initilaize random hyperparameters
#         random_parameters = {K: random.sample(V, 1)[0] for K, V in parameters_grid.items()}
#         #create model with random hyperparameters
#         model = create_model(random_parameters)
#         opt = create_RMSprop_optimizer(random_parameters)
#         #compile model
#         model.compile(optimizer = opt, loss='categorical_crossentropy', metrics=['accuracy'])
#         #train model
#         epochs = 40
#         checkpointer = ModelCheckpoint(filepath='saved_models/model_weights.hdf5', verbose=1, save_best_only=True)
#         early_stopping = EarlyStopping(monitor = 'val_loss', mode = 'min', verbose = 1, patience = 8)
#         history = model.fit(X_train, Y_train,
#                   callbacks = [checkpointer, early_stopping],
#                   validation_split = 0.33,
#                   epochs=epochs, batch_size = 20, verbose=1)
#         #compare current min validation loss with global min validation loss
#         current_min_val_loss = np.min(history.history['val_loss'])
#         if(current_min_val_loss < global_min_val_loss):
#             global_min_val_loss = current_min_val_loss
#             #save the best model so far
#             model.load_weights('saved_models/model_weights.hdf5')
#             model.save('saved_models/best_model.h5')
#             print('saving new best model, val_loss = {}'.format(current_min_val_loss))
#             #save best hyperparameters
#             best_hyperparameters = random_parameters